<a href="https://colab.research.google.com/github/nsp8/HexGridGenerator/blob/master/HexGridGeneratorNew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HexGrid Generator
## Create hexagonal maps representing countries of the world by reading files from your Google Drive

---
Note: In your Google Drive, under ***My Drive***, create a folder **World HexGrid** and in that - an ***Input*** and an ***Output*** folder. Store all your files that need to be processed in the ***Input*** folder.

### Install all the required packages

In [0]:
!pip install geojson geopandas

### Enable Matplotlib visualizations in this notebook

In [0]:
%matplotlib inline

### Import the packages needed to process and create HexGrid maps

In [0]:
from functools import reduce
from google.colab import drive
from numpy import nan
import geojson
import geopandas as gpd
import json
import math
import matplotlib.pyplot as plt
import os
import pandas as pd
import re

### Initialize the global variables - input and output directories
---
*Note: these directories must be in your Google Drive*

In [0]:
input_directory = '/gdrive/My Drive/World HexGrid/Input/'
output_directory = '/gdrive/My Drive/World HexGrid/Output/'

## All the functions that process, clean, format, and visualize your data are defined below:
---

### get_regular_hex_coords
returns the coordinates of the hexagon around ***x*** and ***y***.

In [0]:
def get_regular_hex_coords(x, y):
    r = 1 / math.sin(math.pi/3)
    coords_ = list()
    n = 6
    base_angle = 360 // n
    for i in range(n):
        relative_angle = ((base_angle * i) + (base_angle // 2)) * (math.pi / 180)
        coord_ = [x + (r * math.cos(relative_angle)), y + (r * math.sin(relative_angle))]
        coords_.append(coord_)
    coords_.append(coords_[0])
    return coords_

### create_hex_grid
returns GeoJSON data (```FeatureCollection```) of the input ***DataFrame*** that contains geographical and statistical data of the countries.

In [0]:
def create_hex_grid(dataframe):
    world_hex = geojson.FeatureCollection(features=[])
    i = 0
    countries = dataframe["id"]
    for country in list(countries):
        params = (dataframe[countries == country].to_dict('records'))[0]
        hex_coords = get_regular_hex_coords(params["x"], params["y"])
        country_hexagon = geojson.Polygon([hex_coords])
        country_feature = geojson.Feature(geometry=country_hexagon, 
                                          properties=params)
        world_hex["features"].append(country_feature)
        i += 1
    return world_hex


### save_geojson
saves the GeoJSON object in the Google Drive ***Output*** folder; returns the total path of the stored file, if it was saved successfully, or **```None```** otherwise.

In [0]:
def save_geojson(geojson_obj, filename):
    out_file = os.path.join(output_directory, filename)
    with open(out_file, "w") as output:
        json.dump(geojson_obj, output, indent=4)
    if os.path.lexists(out_file):
        return out_file
    return None

### check_iso_values
returns the name of the column whose values are consistent with ISO2 format in **```data_df```**.

In [0]:
def check_iso_values(data_df):
    # col_map = {}
    for col in data_df.columns:
        values = list(data_df[col])
        string_bool = [is_str for is_str in map(
            lambda x: len(x) == 2 if (type(x) == str) else False, values)]
        all_string = reduce(lambda x, y: x & y, string_bool)
        # col_map[col] = [string_bool.index(b) for b in string_bool if not b]
        if all_string:
            return col
    # print(col_map)
    print("Processed all columns but no match found")
    return None


### remove_spaced_columns
returns a new dataframe with updated column names, if they contain spaces in **```df```**.

In [0]:
def remove_spaced_columns(df):
    new_cols = dict()
    for col in df.columns:
        if len(col) > len(col.strip()):
            # print("{} has to be corrected".format(col))
            new_cols.update({col: col.strip()})
    if new_cols:
        return df.rename(columns=new_cols)
    else:
        return df

### handle_excel_data
returns a single dataframe of the values from all the sheets of the Excel workbook (**```excel_collection```**) -  combined on the basis of the ***country codes (ISO2)***.

In [0]:
def handle_excel_data(excel_collection):
    new_df = pd.DataFrame()
    prev_sheet = ''
    for sheet_name, inputs_df in excel_collection.items():
        inputs_df = remove_spaced_columns(inputs_df)
        inputs_df.replace(r'^\s*$', nan, regex=True, inplace=True)
        inputs_df.replace(r'[-–]+$', nan, regex=True, inplace=True)
        inputs_df.replace(r',', '', regex=True, inplace=True)
        inputs_df.dropna(subset=inputs_df.columns[1:], how='all', inplace=True)
        inputs_df.dropna(axis=1, how="all", inplace=True)
        if new_df.empty:
            new_df = inputs_df
            prev_sheet = sheet_name
            iso_col = check_iso_values(new_df)
            if iso_col:
                new_df.set_index(iso_col, inplace=True)
        else:
            iso_col = check_iso_values(inputs_df)
            if iso_col:
                inputs_df_n = inputs_df.set_index(iso_col)
                new_df = new_df.join(inputs_df_n, 
                                     how="inner",
                                     lsuffix='_{}'.format(prev_sheet),
                                     rsuffix='_{}'.format(sheet_name))
    return new_df

### handle_csv_data
returns a single dataframe of the values from all the sheets of the CSV file (**```data_file```**) -  combined on the basis of the ***country codes (ISO2)***.

In [0]:
def handle_csv_data(data_file, skip=0):
    encodings = ["utf-8", "cp1252", "ISO-8859-1"]
    data_df = pd.DataFrame()
    for encoding in encodings:
        # to handle UnicodeDecodeError
        try:
            if skip:
                data_df = pd.read_csv(data_file, 
                                      encoding=encoding,
                                      keep_default_na=False,
                                      skiprows=list(range(skip)))
            else:
                data_df = pd.read_csv(data_file,
                                      keep_default_na=False,
                                      encoding=encoding)
            break
        except UnicodeDecodeError as e:
            print("Encoding format '{}' didn't work".format(encoding))
            print("Trying the next encoding...\n")
            continue
    print("File read successfully")
    data_df.replace(r'^\s*$', nan, regex=True, inplace=True)
    data_df.replace(r'[-–]+$', nan, regex=True, inplace=True)
    data_df.replace(r',', '', regex=True, inplace=True)
    data_df.dropna(axis=0, how="all", subset=data_df.columns[1:], inplace=True)
    data_df.dropna(axis=1, how="all", inplace=True)
    data_df = remove_spaced_columns(data_df)
    return data_df


In [0]:
encodings = ["utf-8", "cp1252", "ISO-8859-1"]
data_df = pd.DataFrame()
for encoding in encodings:
    # to handle UnicodeDecodeError
    try:
        if skip:
            data_df = pd.read_csv(data_file, 
                                    encoding=encoding,
                                    keep_default_na=False,
                                    skiprows=list(range(skip)))
        else:
            data_df = pd.read_csv(data_file,
                                    keep_default_na=False,
                                    encoding=encoding)
        break
    except UnicodeDecodeError as e:
        print("Encoding format '{}' didn't work".format(encoding))
        print("Trying the next encoding...\n")
        continue

### get_data_df
returns ***DataFrame*** of the files selected (**```file_name```**) from ***Input*** folder.

In [0]:
def get_data_df(file_name):
    data_file = os.path.join(input_directory, "Data", file_name)
    data_df = pd.DataFrame()
    if os.path.lexists(data_file):
        if os.path.isfile(data_file):
            file_ext = file_name.split(".")[-1]
            
            skip_msg = "Do you want to skip any rows (excluding the header)?"
            skip_msg += "\nEnter: y or yes if the rows need to be skipped:\n"
            if file_ext == "csv":
                skip = input(skip_msg)
                if skip and skip.lower() in ["y", "yes"]:
                    skip_rows = int(input(
                        "Enter the number of rows you want to skip:\n"))
                    data_df = handle_csv_data(data_file, skip=skip_rows)
                else:
                    data_df = handle_csv_data(data_file)
                iso_col = check_iso_values(data_df)
                if iso_col:
                    data_df.set_index(iso_col, inplace=True)
            elif file_ext in ["xls", "xlsx"]:
                skip = input("Do you want to skip any rows? \n")
                
                if skip and skip.lower() in ["y", "yes"]:
                    skip_rows = int(input(
                        "Enter the number of rows you want to skip:\n"))
                    data_df = pd.read_excel(data_file,
                                            encoding="utf-8",
                                            sheet_name=None,
                                            keep_default_na=False,
                                            skiprows=list(range(skip_rows)))
                else:
                    data_df = pd.read_excel(data_file,
                                            encoding="utf-8",
                                            sheet_name=None,
                                            keep_default_na=False)
                data_df = handle_excel_data(data_df)
            else:
                print("Invalid File Extension. Please try again")
        else:
            print("Input is not a file")
    else:
        print("File doesnot exist!")
    return data_df

### prompt_column
prompts the user for a column value in the **```dataframe```** to show in the grid.

In [0]:
def prompt_column(dataframe):
    column = input(
        "Which one of these columns do you want to show?: \n{}\n".format(
        list(dataframe.columns)))
    return column

### show_hexgrid, set_plot_properties
reads the saved **```geofile```** from the ***Output*** folder and plots the HexGrid.
**```show_hexgrid```** expects the following arguments: 
1. column name to show 
2. colour_palette values (described here: [Matplotlib Colormaps](https://matplotlib.org/users/colormaps.html))


In [0]:
def set_plot_properties(ax):
    ax.yaxis.get_major_formatter().set_useOffset(False)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.text(s='Hexmap', 
            x=-1.0, y=-3.5, verticalalignment='bottom', 
            horizontalalignment='left')


def show_hexgrid(geo_file, color_palette="blues", column=None, col_name=None):
    if geo_file:
        plt.rcParams.update({'font.size': 22})
        plt.rcParams["axes.formatter.useoffset"] = False
        worldhex_df = gpd.read_file(geo_file)
        column = None if column == "geometry" or column == '' else column
        
        ax = worldhex_df.plot(column=column, 
                              cmap=color_palette,
                              linewidth=3,
                              edgecolor='black',
                              figsize=(24, 24))

        for ix, row in worldhex_df.iterrows():
            x, y = row["x"], row["y"]
            ax.annotate(row['id'], xy=(x, y), 
                        horizontalalignment='center',
                        verticalalignment='center')
        if column:
            fig = ax.get_figure()
            cax = fig.add_axes([0.90, 0.62, 0.008, 0.15])
            vmin, vmax = worldhex_df[column].min(), worldhex_df[column].max()
            sm = plt.cm.ScalarMappable(cmap=color_palette, 
                                       norm=plt.Normalize(vmin=vmin, vmax=vmax))
            sm._A = []
            cbar = fig.colorbar(sm, cax=cax)
            if col_name:
                cbar.set_label(col_name, labelpad=25)
            else:
                cbar.set_label(column, labelpad=25)
        plt.tight_layout()
        set_plot_properties(ax)


## Mount the Google Drive to use it in this Notebook
---
### *Requires Authentication*
To allow Google Colab to access your Google Drive, follow the instructions given in the cell below:


1.   Click on the link below which will ask you to select your Google account
2.   Authenticate your account. You will be redirected to a screen with a security key
3.   Copy the security key and paste it into the box in the cell below

In [0]:
drive.mount('/gdrive')

## Read the HexCodes file from ***Input*** folder
---
*Note: This cell expects this file to be in CSV format.*

In [0]:
hexcodes_list = os.listdir(os.path.join(input_directory, "Hexcodes"))
hexcodes_filename = ""
if hexcodes_list:
    hexcodes_filename = input(
        "Enter the hexcodes file name - \nOptions: {}\n".format(
        hexcodes_list))
    print("Reading {} ...".format(hexcodes_filename))
else:
    print("No hexcode file present! \nPlease upload a file in the Input folder and try again.")

In [0]:
if not hexcodes_filename:
    hexcodes_filename = "hexcodes_expanded.csv"
hexcode_file = os.path.join(input_directory, "Hexcodes", hexcodes_filename)
hexcodes = pd.read_csv(hexcode_file, encoding="UTF-8")

## Visualize the *HexCodes* file.
---

In [0]:
column = None
world_hex = create_hex_grid(hexcodes)
world_hex_path = save_geojson(world_hex, "new_output.geojson")
show_hexgrid(world_hex_path, "plasma", column)

## Enter the file name you want to combine with the HexCodes:
---
*Note: Don't include quotes in the file name.*

In [0]:
data_files = os.listdir(os.path.join(input_directory, "Data"))
data_file = ""
if data_files:
    data_file = input(
        "Enter the file name you want to combine - \nOptions: {}\n".format(
        data_files))
    print("\nReading {} ...".format(data_file))
else:
    print("No data file to integrate! \nPlease upload a file in the Input folder and try again.")

## Create a DataFrame from the selected file from Google Drive.
---

In [0]:
data_df = get_data_df(data_file)

## Combine HexCodes with the input data file.
---

In [0]:
if not data_df.empty:
    hexcode_df = hexcodes.set_index("id")
    combi_df = hexcode_df.join(data_df, how="inner")
    combi_df.reset_index(inplace=True)
    combi_df.rename(columns={'index': 'id'}, inplace=True)
    combi_df.dropna(how="all", axis=1, inplace=True)
    combi_df.fillna('', inplace=True)

## Clean the DataFrame
---
*gets rid of duplicates and empty columns.*

In [0]:
if not data_df.empty:
    combi_new_df = combi_df.T.drop_duplicates().T

## Read the name of the column required to display on the plot:
---
*Note: Don't include quotes in the column name.*

In [0]:
if not data_df.empty:
    column = prompt_column(combi_new_df)

## Display name of this column:
---
*Enter the name to associate the column with, that'll be displayed on the chart*

In [0]:
column_display_name = None
if column:
    column_display_name = input("Enter the name to display on the chart:\n")
    if not column_display_name:
        column_display_name = column
else:
    print("No column was selected in the previous step.")

## Create a filtered data-set on the column defined above.
---
*Criteria: filters values of the column in the range of **x**.*

In [0]:
def validate_min_max(min_val, max_val):
    bools = [True, True]
    try:
        if not min_val.isalpha() and not max_val.isalpha():
            if float(max_val) <= float(min_val):
                print("\nMinimum value can't be greater than the maximum value!")
                print("Assigning the default min-max values")
                bools = [False, False]
    except ValueError:
        bools = [False, False]
    return bools


In [0]:
if not data_df.empty:
    combi_new_df[column] = combi_new_df[column].replace(
        r'^\s*$', nan, regex=True).dropna()
    combi_new_df[column] = combi_new_df[column].astype("float")
    x_min = input(
        "Enter the minimum value of the column '{}'\n".format(column))
    x_max = input(
        "Enter the maximum value of the column '{}'\n".format(column))
    col_min, col_max = (combi_new_df[column].min(), combi_new_df[column].max())
    if (not x_min) or (x_min.isalpha()):
        x_min = str(col_min)
    if (not x_max) or (x_max.isalpha()):
        x_max = str(col_max)
    min_valid, max_valid = validate_min_max(x_min, x_max)
    combi_x_min = float(col_min) if not min_valid else float(x_min)
    combi_x_max = float(col_max) if not max_valid else float(x_max)
    print("\nMinimum value: {}\nMaximum value: {}".format(combi_x_min, 
                                                          combi_x_max))
    filter_condition = combi_new_df[column].ge(combi_x_min) & \
                       combi_new_df[column].le(combi_x_max)
    subset_df = combi_new_df[filter_condition]

## Read the colormap values:
---
All the colormaps are here: [Matplotlib Colormaps](https://matplotlib.org/users/colormaps.html)

*Note: Don't include quotes in the colormap names.*

In [0]:
if not data_df.empty:
    colormap = input(
        "Enter the file name you want to combine - \nOptions: {}\n".format(
        plt.colormaps()))

## Create the visualization of the data:
---

In [0]:
if not data_df.empty:
    if not subset_df.empty:
        combi_hex = create_hex_grid(subset_df)
        out_file = save_geojson(combi_hex, "combi_hex.geojson")
        show_hexgrid(out_file, colormap, column, column_display_name)
    else:
        print("No data found in this range. Plotting for all values:")
        combi_hex = create_hex_grid(combi_new_df)
        out_file = save_geojson(combi_hex, "combi_hex.geojson")
        show_hexgrid(out_file, colormap, column, column_display_name)